In [7]:
import tensorflow_datasets as tfds

def load_dataset(ds_language_code):
    ds = tfds.load('wiki40b/' + ds_language_code, split='test')
    wikis = list(ds.as_numpy_iterator())
    return wikis

In [8]:
import re

def _remove_matetags(raw_text):
    SA = '_START_ARTICLE_'
    SS = '_START_SECTION_'
    SP = '_START_PARAGRAPH_'
    NL = '_NEWLINE_'
    META_TAGS = [SA, SS, SP, NL]
    for meta_tag in META_TAGS:
        raw_text = raw_text.replace(meta_tag, '')
    return raw_text

  
def preprocess(wikis):
    texts = []
    for wiki in wikis:
        
        raw_text = wiki['text'].decode()
        text = _remove_matetags(raw_text)
        texts.append(text)
    
    sentences = []
    for text in texts:
        sentences += re.split('[.\n]', text)
    # remove empty sentence
    sentences = list(filter(lambda x: x != '', sentences))
    return sentences


In [9]:
import nltk
nltk.download('all')

CONJUNCTION = 'CC'
PREPOSITION = 'IN'

def _count_next_word(word, sentences):
  count_dict = {}
  for sentence in sentences:
    words = sentence.split()
    if word in words:
      index = words.index(word)
      try:
        next_word = words[index + 1]
      except IndexError:
        continue
      if next_word not in count_dict:
        count_dict[next_word] = 1
      else:
        count_dict[next_word] += 1
  return count_dict
  
#descending order
def _sort_dict_by_value(dic):
  dic = sorted(dic.items(), key = lambda x : x[1] ,reverse=True)
  return dic

def get_next_word(word, sentences):
  count_dict = _count_next_word(word, sentences)
  count_dict = _sort_dict_by_value(count_dict)
  for d in count_dict:
    word = d[0]
    morph = nltk.word_tokenize(word)
    pos = nltk.pos_tag(morph)
    word_class = pos[0][1]
    if word_class != CONJUNCTION and word_class != PREPOSITION:
      return word


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

In [10]:
# TODO clean
def _count_previous_word(word, next_word, sentences):
  count_dict = {}
  for sentence in sentences:
    words = sentence.split()
    if word in words:
      index = words.index(word)
      try:
        previous_word = words[index - 1]
        if words[index + 1] == next_word:
          if next_word not in count_dict:
            count_dict[previous_word] = 1
          else:
            count_dict[previous_word] += 1
      except IndexError:
        continue
      
  return count_dict

def get_previous_word(word, next_word, sentences):
  count_dict = _count_previous_word(word, next_word, sentences)
  count_dict = _sort_dict_by_value(count_dict)
  previous_word = count_dict[0][0]
  return previous_word

In [11]:

def main():
  word = 'apple'
  ds_language = 'en'
  wikis = load_dataset(ds_language)
  sentences = preprocess(wikis)
  next_word = get_next_word(word, sentences)
  speech_manuscript = word + ' ' + next_word
  print(speech_manuscript)
  # previous_word = get_previous_word(word, next_word, sentences)
  # print(previous_word)
  

if __name__ == '__main__':
    main()

trees
[('of', 1), ('to', 1), ('several', 1), ('pink', 1), ('quality', 1), ('finds', 1), ('the', 1), ('bringing', 1), ("Garrison's", 1), ('and', 1), ('propagate', 1), ('cleared,', 1), ('many', 1), ('for', 1), ('avoid', 1)]
of
